In [37]:
# Import the packages the project required
from bs4 import BeautifulSoup
import requests
import csv
import numpy as np
import pandas as pd
from pandas import DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium
import random 
from IPython.display import Image 
from IPython.core.display import HTML

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.19.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [50]:
# Import data with all UK's postcode, latitude and longitude from web
rawdata = pd.read_csv('https://raw.githubusercontent.com/Gibbs/uk-postcodes/master/postcodes.csv')

# Drop the columns we dont need
rawdata = rawdata.drop('eastings', 1)
rawdata = rawdata.drop('northings', 1)
rawdata = rawdata.drop('region', 1)
rawdata = rawdata.drop('uk_region', 1)
rawdata = rawdata.drop('country', 1)
rawdata = rawdata.drop('country_string', 1)
rawdata = rawdata.drop('town', 1)

# Import data with London's Borough, Neighborhood and PostCode from local csv
rawdata2 = {'Borough':["Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Eastern Central","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","Northern","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","North Western","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Eastern","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","South Western","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Battersea","Western","Western","Western","Western","Western","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Paddington","Western Central","Western Central","Western Central","Western Central","Western Central","Western Central","Western Central","Western Central","Western Central"],'PostCode':["E1","E1","E1","E1","E1","E1","E1","E2","E2","E2","E3","E3","E3","E3","E3","E3","E4","E4","E4","E5","E5","E6","E6","E6","E7","E7","E7","E7","E7","E8","E8","E8","E9","E9","E9","E10","E11","E11","E11","E12","E12","E12","E13","E13","E14","E14","E14","E14","E15","E15","E15","E15","E15","E15","E16","E16","E16","E16","E16","E16","E16","E17","E17","E18","E18","E18","EC1","EC1","EC1","EC1","EC1","EC1","EC2","EC2","EC2","EC2","EC2","EC2","EC3","EC3","EC3","EC3","EC3","EC3","EC4","EC4","EC4","EC4","EC4","N1","N1","N1","N1","N1","N1","N1","N2","N2","N3","N3","N3","N4","N4","N4","N4","N4","N5","N5","N5","N6","N6","N6","N6","N7","N7","N8","N8","N8","N8","N9","N10","N10","N10","N11","N11","N12","N12","N13","N14","N14","N14","N15","N15","N16","N16","N16","N16","N17","N18","N19","N19","N19","N19","N19","N20","N20","N21","N22","N22","N22","NW1","NW1","NW1","NW1","NW2","NW2","NW2","NW2","NW2","NW2","NW2","NW3","NW3","NW3","NW3","NW3","NW3","NW3","NW4","NW4","NW5","NW5","NW5","NW5","NW6","NW6","NW6","NW6","NW6","NW6","NW6","NW7","NW8","NW8","NW8","NW9","NW9","NW9","NW9","NW10","NW10","NW10","NW10","NW10","NW10","NW10","NW11","NW11","NW11","NW11","SE1","SE1","SE1","SE1","SE1","SE1","SE1","SE2","SE2","SE2","SE3","SE3","SE3","SE3","SE3","SE4","SE4","SE4","SE4","SE4","SE5","SE5","SE5","SE6","SE6","SE6","SE6","SE7","SE7","SE8","SE8","SE8","SE8","SE9","SE9","SE9","SE9","SE10","SE10","SE11","SE11","SE11","SE12","SE12","SE12","SE12","SE12","SE12","SE13","SE13","SE13","SE14","SE14","SE14","SE14","SE15","SE15","SE15","SE15","SE16","SE16","SE16","SE16","SE16","SE16","SE16","SE16","SE17","SE17","SE17","SE17","SE17","SE18","SE18","SE18","SE18","SE19","SE19","SE19","SE19","SE19","SE19","SE19","SE20","SE20","SE20","SE21","SE21","SE21","SE21","SE22","SE22","SE22","SE23","SE23","SE24","SE24","SE24","SE25","SE25","SE25","SE26","SE26","SE26","SE26","SE26","SE27","SE27","SE27","SE28","SW1","SW1","SW1","SW1","SW1","SW1","SW1","SW2","SW2","SW2","SW2","SW3","SW3","SW3","SW4","SW4","SW4","SW5","SW5","SW5","SW6","SW6","SW6","SW7","SW7","SW7","SW7","SW8","SW8","SW8","SW8","SW8","SW9","SW9","SW9","SW9","SW10","SW10","SW10","SW10","SW10","SW11","SW11","SW12","SW12","SW12","SW13","SW13","SW14","SW14","SW15","SW15","SW15","SW15","SW15","SW15","SW16","SW16","SW16","SW16","SW16","SW16","SW17","SW17","SW17","SW17","SW17","SW18","SW18","SW18","SW19","SW19","SW19","SW19","SW19","SW20","SW20","SW20","SW20","SW20","W1","W1","W1","W1","W1","W2","W2","W2","W2","W2","W2","W3","W4","W4","W5","W6","W6","W6","W7","W8","W9","W9","W9","W9","W10","W10","W10","W10","W10","W10","W11","W11","W11","W11","W11","W12","W12","W13","W14","W14","W14","WC1","WC1","WC1","WC1","WC1","WC2","WC2","WC2","WC2"],'Neighborhood':["Aldgate","Bethnal Green","City of London","Mile End","Shoreditch","Stepney","Whitechapel","Bethnal Green","Shoreditch","Stepney","Bethnal Green","Bow","Bromley by Bow","Poplar","Stepney","West Ham","Chingford","Highams Park","Stepney","Clapton","Hackney","Barking","Beckton","East Ham","East Ham","Forest Gate","Upton Park","Wanstead","West Ham","Dalston","Hackney","Shoreditch","Hackney","Homerton","Poplar","Leyton","Leyton","Leytonstone","Wanstead","East Ham","Manor Park","Wanstead","Plaistow","West Ham","Isle of Dogs","Millwall","Poplar","Stepney","Canning Town","Hackney","Leyton","Poplar","Stratford","West Ham","Canning Town","Docklands","East Ham","North Woolwich","Victoria Dock","West Ham","Woolwich","Leyton","Walthamstow","South Woodford","Wanstead","Woodford","Barbican","City of London","Clerkenwell","Finsbury","Holborn","Shoreditch","Bishopsgate","City of London","Finsbury","Liverpool Street","Moorgate","Shoreditch","Aldgate","City of London","Fenchurch Street","Monument","Stepney","Tower Hill","Blackfriars","City of London","Fleet Street","St. Paul's","Temple","Barnsbury","Canonbury","Finsbury","Hackney","Islington","Shoreditch","St. Pancras","East Finchley","Hornsey","Finchley","Finchley Central","Finchley Church End","Finsbury Park","Hornsey","Islington","Manor House","Stoke Newington","Highbury","Islington","Tottenham","Hampstead","Highgate","Hornsey","St. Pancras","Holloway","Islington","Crouch End","Hornsey","Islington","Tottenham","Lower Edmonton","Hornsey","Muswell Hill","Wood Green","Friern Barnet","New Southgate","North Finchley","Woodside Park","Palmers Green","Islington","Southgate","Tottenham","Seven Sisters","South Tottenham","Hackney","Islington","Stamford Hill","Stoke Newington","Tottenham","Upper Edmonton","Archway","Islington","St. Pancras","Tufnell Park","Upper Holloway","Totteridge","Whetstone","Winchmore Hill","Alexandra Palace/Park","Tottenham","Wood Green","Camden Town","Regent's Park","St. Marylebone","St. Pancras","Cricklewood","Dollis Hill","Hampstead","Hendon","Neasden","Willesden","Willesden Green","Belsize Park","Brent Cross","Finchley","Hampstead","Hendon","St. Pancras","Swiss Cottage","Brent Cross","Hendon","Hampstead","Hendon","Kentish Town","St. Pancras","Brondesbury Park","Kilburn","Paddington","Queens Park","South Hampstead","West Hampstead","Willesden","Mill Hill","Hampstead","St. John's Wood","St. Marylebone","Colindale","Hendon","Kingsbury","The Hyde","Acton","Hammersmith","Harlesden","Kensal Green","Kensal Rise","Wembley","Willesden","Finchley","Golders Green","Hampstead Garden Suburb","Hendon","Bermondsey","Borough","Camberwell","Lambeth","Southwark","Waterloo","Woolwich","Abbey Wood","Erith","Woolwich","Blackheath","Greenwich","Kidbrooke","Lewisham","Westcombe Park","Brockley","Crofton Park","Deptford","Honor Oak Park","Lewisham","Camberwell","Lambeth","Southwark","Bellingham","Catford","Hither Green","Lewisham","Charlton","Greenwich","Bermondsey","Deptford","Greenwich","Lambeth","Chislehurst","Eltham","Mottingham","Woolwich","Greenwich","Lewisham","Kennington","Lambeth","Southwark","Chislehurst","Grove Park","Lambeth","Lee","Lewisham","Woolwich","Greenwich","Hither Green","Lewisham","Camberwell","Deptford","New Cross","New Cross Gate","Camberwell","Deptford","Nunhead","Peckham","Bermondsey","Camberwell","Deptford","Lewisham","Rotherhithe","South Bermondsey","Surrey Docks","Woolwich","Camberwell","Elephant & Castle","Lambeth","Southwark","Walworth","Greenwich","Lewisham","Plumstead","Woolwich","Camberwell","Croydon","Crystal Palace","Lambeth","Lewisham","Penge","Upper Norwood","Anerley","Beckenham","Penge","Camberwell","Dulwich","Lambeth","West Dulwich","Camberwell","East Dulwich","Wandsworth","Forest Hill","Lewisham","Camberwell","Herne Hill","Lambeth","Croydon","Lambeth","South Norwood","Beckenham","Camberwell","Lewisham","Sydenham","Upper Sydenham","Lambeth","Tulse Hill","West Norwood","Thamesmead","Battersea","Belgravia","Brompton","Chelsea","Pimlico","Victoria","Westminster","Brixton","Lambeth","Streatham Hill","Wandsworth","Brompton","Chelsea","Kensington","Clapham","Lambeth","Wandsworth","Earl's Court","Fulham","Kensington","Fulham","Parson's Green","Wandsworth","Brompton","Knightsbridge","South Kensington","Westminster","Battersea","Nine Elms","South Lambeth","Vauxhall","Wandsworth","Brixton","Lambeth","North Brixton","Stockwell","Chelsea","Fulham","Kensington","West Brompton","World's End","Battersea","Clapham Junction","Balham","Battersea","Wandsworth","Barnes","Castelnau","East Sheen","Mortlake","Barnes","Kensington","Malden","Putney","Roehampton","Wandsworth","Croydon","Lambeth","Mitcham","Norbury","Streatham","Wandsworth","Battersea","Mitcham","Tooting","Wandsworth","Wimbledon","Battersea","Earlsfield","Wandsworth","Collier's Wood","Merton","Mitcham","Wandsworth","Wimbledon","Cottenham Park","Merton","Raynes Park","West Wimbledon","Wimbledon","Mayfair","Soho","St. Marylebone","St. Pancras","Westminster","Acton","Bayswater","Hyde Park","Kensington","Paddington","Westminster","Acton","Acton","Chiswick","Ealing","Chiswick","Fulham","Hammersmith","Hanwell","Kensington","Maida Hill","Maida Vale","Paddington","Warwick Avenue","Hammersmith","Kensington","Ladbroke Grove","North Kensington","Paddington","Willesden","Hammersmith","Holland Park","Kensington","Notting Hill","Paddington","Hammersmith","Shepherd's Bush","West Ealing","Fulham","Hammersmith","West Kensington","Bloomsbury","Finsbury","Gray's Inn","Holborn","St. Pancras","Covent Garden","Holborn","Strand","Westminster"]}
rawdata2 = pd.DataFrame(rawdata2)
rawdata3 = {'Borough':["","","","","","","",""],'PostCode':["","","","","","","",""],'Neighborhood':["","","","","","","",""]}
rawdata3 = pd.DataFrame(rawdata3)
rawdata2=rawdata2.append(rawdata3,ignore_index=True)

# Add Neighborhoods with same PostCode in one line
a=[]
for j in range(len(rawdata2["PostCode"])-8):
        if rawdata2["PostCode"][j] == rawdata2["PostCode"][j+1]:
            rawdata2["Neighborhood"][j] = rawdata2["Neighborhood"][j]+','+rawdata2["Neighborhood"][j+1]
            if rawdata2["PostCode"][j] == rawdata2["PostCode"][j+2]:
                rawdata2["Neighborhood"][j] = rawdata2["Neighborhood"][j]+','+rawdata2["Neighborhood"][j+2]    
                if rawdata2["PostCode"][j] == rawdata2["PostCode"][j+3]:
                    rawdata2["Neighborhood"][j] = rawdata2["Neighborhood"][j]+','+rawdata2["Neighborhood"][j+3]
                    if rawdata2["PostCode"][j] == rawdata2["PostCode"][j+4]:
                        rawdata2["Neighborhood"][j] = rawdata2["Neighborhood"][j]+','+rawdata2["Neighborhood"][j+4]
                        if rawdata2["PostCode"][j] == rawdata2["PostCode"][j+5]:
                            rawdata2["Neighborhood"][j] = rawdata2["Neighborhood"][j]+','+rawdata2["Neighborhood"][j+5]
                            if rawdata2["PostCode"][j] == rawdata2["PostCode"][j+6]:
                                rawdata2["Neighborhood"][j] = rawdata2["Neighborhood"][j]+','+rawdata2["Neighborhood"][j+6]
                                if rawdata2["PostCode"][j] == rawdata2["PostCode"][j+7]:
                                    rawdata2["Neighborhood"][j] = rawdata2["Neighborhood"][j]+','+rawdata2["Neighborhood"][j+7]
                                    if rawdata2["PostCode"][j] == rawdata2["PostCode"][j+8]:
                                        rawdata2["Neighborhood"][j] = rawdata2["Neighborhood"][j]+','+rawdata2["Neighborhood"][j+8]
            a.append(j+1)
rawdata2=rawdata2.drop(a)
rawdata2.reset_index(inplace=True)
rawdata2=rawdata2.drop(columns=["index"])
rawdata2=rawdata2.drop([119,120,121,122,123,124,125,126])
rawdata2.reset_index(inplace=True)
rawdata2=rawdata2.drop(columns=["index"])

# Merge datasets with key "PostCode", the new data will be called "london"
PostCode = [None]*119
Borough = [None]*119
Neighborhood = [None]*119
Latitude = [None]*119
Longitude = [None]*119
for a in range(119):
    PostCode[a] = rawdata2['PostCode'][a]
    Borough[a] = rawdata2['Borough'][a]
    Neighborhood[a] = rawdata2['Neighborhood'][a]
for b in range(2855):
    for a in range(119):
        if rawdata['postcode'][b] == rawdata2['PostCode'][a]:
            Latitude[a] = rawdata['latitude'][b]
            Longitude[a] = rawdata['longitude'][b]           
csv_file = open('Neighborhood in London.csv','w')     
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['PostCode','Borough','Neighborhood','Latitude','Longitude'])
for i in range(119):
    csv_writer.writerow([PostCode[i],Borough[i],Neighborhood[i],Latitude[i],Longitude[i]])
csv_file.close()
london = pd.read_csv("Neighborhood in London.csv")

# Change the order of columns
cols = list(london)
cols.insert(0,cols.pop(cols.index('Neighborhood')))
cols.insert(0,cols.pop(cols.index('Borough'))) 
london = london.loc[:,cols]
london = london.dropna(axis=1)

In [52]:
# Grab rawdata from Wiki
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
body = soup.find('tbody').text
body = body.split('\n')

# Select Toronto's Postcode, Borough and Neighborhood from rawdata
PostCode=[None]*288
Borough=[None]*288
Neighborhood=[None]*288
for a in range(288):
    PostCode[a] = body[6+a*5]
for b in range(288):    
    Borough[b] = body[7+b*5]    
for c in range(288):
    Neighborhood[c] = body[8+c*5]
csv_file = open('Neighborhood in Toronto.csv','w')     
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['PostCode','Borough','Neighborhood'])
for i in range(288):
    csv_writer.writerow([PostCode[i],Borough[i],Neighborhood[i]])
csv_file.close()
rawdata4 = pd.read_csv("Neighborhood in Toronto.csv")

# Remove the Unassigned Borough from dataset
for i in range(288):
    if rawdata4["Borough"][i]=='Not assigned':
        rawdata4["Borough"][i]='0'
a=rawdata4[(rawdata4.Borough=='0')].index.tolist()
rawdata4=rawdata4.drop(a)
rawdata4.reset_index(inplace=True)
rawdata4=rawdata4.drop(columns=["index"])

# Add Neighborhoods with same PostCode in one line
rawdata3 = {'Borough':["","","","","","","",""],'PostCode':["","","","","","","",""],'Neighborhood':["","","","","","","",""]}
rawdata3 = pd.DataFrame(rawdata3)
rawdata4=rawdata4.append(rawdata3,ignore_index=True)
a=[]
for j in range(len(rawdata4["PostCode"])-8):
        if rawdata4["PostCode"][j] == rawdata4["PostCode"][j+1]:
            rawdata4["Neighborhood"][j] = rawdata4["Neighborhood"][j]+','+rawdata4["Neighborhood"][j+1]
            if rawdata4["PostCode"][j] == rawdata4["PostCode"][j+2]:
                rawdata4["Neighborhood"][j] = rawdata4["Neighborhood"][j]+','+rawdata4["Neighborhood"][j+2]    
                if rawdata4["PostCode"][j] == rawdata4["PostCode"][j+3]:
                    rawdata4["Neighborhood"][j] = rawdata4["Neighborhood"][j]+','+rawdata4["Neighborhood"][j+3]
                    if rawdata4["PostCode"][j] == rawdata4["PostCode"][j+4]:
                        rawdata4["Neighborhood"][j] = rawdata4["Neighborhood"][j]+','+rawdata4["Neighborhood"][j+4]
                        if rawdata4["PostCode"][j] == rawdata4["PostCode"][j+5]:
                            rawdata4["Neighborhood"][j] = rawdata4["Neighborhood"][j]+','+rawdata4["Neighborhood"][j+5]
                            if rawdata4["PostCode"][j] == rawdata4["PostCode"][j+6]:
                                rawdata4["Neighborhood"][j] = rawdata4["Neighborhood"][j]+','+rawdata4["Neighborhood"][j+6]
                                if rawdata4["PostCode"][j] == rawdata4["PostCode"][j+7]:
                                    rawdata4["Neighborhood"][j] = rawdata4["Neighborhood"][j]+','+rawdata4["Neighborhood"][j+7]
                                    if rawdata4["PostCode"][j] == rawdata4["PostCode"][j+8]:
                                        rawdata4["Neighborhood"][j] = rawdata4["Neighborhood"][j]+','+rawdata4["Neighborhood"][j+8]
            a.append(j+1)
rawdata4=rawdata4.drop(a)
rawdata4.reset_index(inplace=True)
rawdata4=rawdata4.drop(columns=["index"])
rawdata4=rawdata4.drop([103,104,105,106,107,108,109,110])
rawdata4.reset_index(inplace=True)
rawdata4=rawdata4.drop(columns=["index"])

# Deal with unassigned Neighborhood data
for k in range(103):
    if rawdata4["Neighborhood"][k]=='Not assigned':
        rawdata4["Neighborhood"][k]=rawdata4["Borough"][k]

# Import data with Latitude and Longitude and merge with dataset, the new data will be called "toronto"
!wget -O Geospatial_data.csv http://cocl.us/Geospatial_data
df = DataFrame(rawdata4)
rawdata4 = df.sort_values(by='PostCode', ascending=True)
rawdata4.reset_index(inplace=True)
rawdata4 = rawdata4.drop(columns=["index"])
rawdata4 = DataFrame(rawdata4)
geodata = pd.read_csv("Geospatial_data.csv")
df2 = DataFrame(geodata)
geodata2 = df2.sort_values(by='Postal Code', ascending=True)
geodata2.reset_index(inplace=True)
geodata2 = geodata2.drop(columns=["index"])
geodata2 = DataFrame(geodata2)
geodata2 = geodata2.rename(columns={'Postal Code':'PostCode'})
toronto = pd.merge(rawdata4,geodata2).dropna(axis=1)

--2019-04-18 09:11:54--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-04-18 09:11:54--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-04-18 09:11:55--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-04-18 09:11:56--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjc

In [53]:
# Download and explore data
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# loop through the data and fill the dataframe one row at a time
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

# Use geopy library to get the latitude and longitude values of New York City    
address = 'New York City, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
newyork = neighborhoods.dropna(axis=1)

In [54]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'YUKN0Q1PDQH3240RJVDFXOTHUVSJWX45FXKJLOFJJRWLQEMM' # your Foursquare ID
CLIENT_SECRET = '03BBZSSXV2FMQLDODD3FRPIT23H5UAXFKQTSODVJVT012EKQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius=500

# Function to repeat the same process to all the neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)      
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']    
    return(nearby_venues)

In [55]:
# run the above function on each neighborhood and create some new dataframe of venues
london_venues = getNearbyVenues(names=london['Neighborhood'],
                                   latitudes=london['Latitude'],
                                   longitudes=london['Longitude']
                                  )
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )
newyork_venues = getNearbyVenues(names=newyork['Neighborhood'],
                                   latitudes=newyork['Latitude'],
                                   longitudes=newyork['Longitude']
                                  )
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
print('There are {} uniques categories.'.format(len(newyork_venues['Venue Category'].unique())))

# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Neighborhood'] = london_venues['Neighborhood'] 
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
newyork_onehot['Neighborhood'] = newyork_venues['Neighborhood'] 

# move neighborhood column to the first column
london_fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[london_fixed_columns]
toronto_fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[toronto_fixed_columns]
newyork_fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[newyork_fixed_columns]

# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
london_grouped = london_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
newyork_grouped = newyork_onehot.groupby('Neighborhood').mean().reset_index()

# each neighborhood along with the top 5 most common venues
num_top_venues = 5
for hood in london_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 1})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 1})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
for hood in newyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = newyork_grouped[newyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 1})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


Aldgate,Bethnal Green,City of London,Mile End,Shoreditch,Stepney,Whitechapel
Bethnal Green,Shoreditch,Stepney
Bethnal Green,Bow,Bromley by Bow,Poplar,Stepney,West Ham
Chingford,Highams Park,Stepney
Clapton,Hackney
Barking,Beckton,East Ham
East Ham,Forest Gate,Upton Park,Wanstead,West Ham
Dalston,Hackney,Shoreditch
Hackney,Homerton,Poplar
Leyton
Leyton,Leytonstone,Wanstead
East Ham,Manor Park,Wanstead
Plaistow,West Ham
Isle of Dogs,Millwall,Poplar,Stepney
Canning Town,Hackney,Leyton,Poplar,Stratford,West Ham
Canning Town,Docklands,East Ham,North Woolwich,Victoria Dock,West Ham,Woolwich
Leyton,Walthamstow
South Woodford,Wanstead,Woodford
Barbican,City of London,Clerkenwell,Finsbury,Holborn,Shoreditch
Bishopsgate,City of London,Finsbury,Liverpool Street,Moorgate,Shoreditch
Aldgate,City of London,Fenchurch Street,Monument,Stepney,Tower Hill
Blackfriars,City of London,Fleet Street,St. Paul's,Temple
Barnsbury,Canonbury,Finsbury,Hackney,Islington,Shoreditch,St. Pancras
East Finchley,Hornsey
F

Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Mor

                 venue  freq
0  Sporting Goods Shop   0.1
1             Platform   0.1
2          Pizza Place   0.1
3                  Bar   0.1
4          Yoga Studio   0.0


----Bermondsey,Deptford,Greenwich,Lambeth----
                   venue  freq
0                    Bar   0.1
1                    Pub   0.1
2          Grocery Store   0.1
3  Vietnamese Restaurant   0.1
4            Coffee Shop   0.1


----Bethnal Green,Bow,Bromley by Bow,Poplar,Stepney,West Ham----
                venue  freq
0                 Pub   0.2
1  Light Rail Station   0.1
2                 Gym   0.1
3        Burger Joint   0.1
4                 Bar   0.1


----Bethnal Green,Shoreditch,Stepney----
             venue  freq
0              Pub   0.1
1     Cocktail Bar   0.1
2      Coffee Shop   0.1
3             Café   0.1
4  Organic Grocery   0.0


----Bishopsgate,City of London,Finsbury,Liverpool Street,Moorgate,Shoreditch----
          venue  freq
0         Hotel   0.1
1    Food Truck   0.1
2   Coffee Shop

                    venue  freq
0       French Restaurant   0.1
1     Indie Movie Theater   0.1
2             Pizza Place   0.1
3                    Café   0.1
4  Okonomiyaki Restaurant   0.0


----Croydon,Lambeth,Mitcham,Norbury,Streatham,Wandsworth----
                 venue  freq
0                  Pub   0.2
1                  Bar   0.1
2     Video Game Store   0.1
3  Fried Chicken Joint   0.1
4            Gastropub   0.1


----Croydon,Lambeth,South Norwood----
               venue  freq
0               Café   0.2
1               Park   0.1
2        Supermarket   0.1
3  Fish & Chips Shop   0.1
4           Platform   0.1


----Dalston,Hackney,Shoreditch----
                  venue  freq
0                Bakery   0.1
1                   Pub   0.1
2                  Café   0.1
3            Restaurant   0.1
4  Gym / Fitness Center   0.1


----Ealing----
                           venue  freq
0                            Pub   0.1
1                    Coffee Shop   0.1
2                 

                     venue  freq
0  Fruit & Vegetable Store   0.1
1     Gym / Fitness Center   0.1
2            Grocery Store   0.1
3           Clothing Store   0.1
4           Sandwich Place   0.1


----Maida Hill,Maida Vale,Paddington,Warwick Avenue----
                venue  freq
0  Italian Restaurant   0.1
1         Pizza Place   0.1
2       Grocery Store   0.1
3         Coffee Shop   0.1
4                Café   0.1


----Mayfair,Soho,St. Marylebone,St. Pancras,Westminster----
                           venue  freq
0                 Clothing Store   0.1
1                    Yoga Studio   0.0
2                    Pastry Shop   0.0
3  Paper / Office Supplies Store   0.0
4           Pakistani Restaurant   0.0


----Mill Hill----
                    venue  freq
0                    Park   1.0
1             Yoga Studio   0.0
2  Okonomiyaki Restaurant   0.0
3    Pakistani Restaurant   0.0
4    Outdoor Supply Store   0.0


----North Finchley,Woodside Park----
              venue  freq
0  

...

                 venue  freq
0        Grocery Store   0.1
1          Supermarket   0.1
2  Fried Chicken Joint   0.1
3   Chinese Restaurant   0.1
4             Pharmacy   0.1


----Fieldston----
         venue  freq
0        River   0.2
1  High School   0.2
2   Playground   0.2
3        Plaza   0.2
4    Piano Bar   0.0


----Financial District----
                           venue  freq
0                    Coffee Shop   0.1
1                    Yoga Studio   0.0
2  Paper / Office Supplies Store   0.0
3                      Piano Bar   0.0
4             Photography Studio   0.0


----Flatbush----
                  venue  freq
0    Mexican Restaurant   0.1
1  Caribbean Restaurant   0.1
2         Deli / Bodega   0.1
3           Coffee Shop   0.1
4           Yoga Studio   0.0


----Flatiron----
                venue  freq
0         Yoga Studio   0.0
1            Pie Shop   0.0
2  Photography Studio   0.0
3            Pharmacy   0.0
4           Pet Store   0.0


----Flatlands----
           

                  venue  freq
0       Harbor / Marina   0.5
1         Moving Target   0.5
2           Yoga Studio   0.0
3  Pakistani Restaurant   0.0
4    Photography Studio   0.0


----Hudson Yards----
                venue  freq
0                Café   0.1
1  Italian Restaurant   0.1
2         Coffee Shop   0.1
3               Hotel   0.1
4         Yoga Studio   0.0


----Huguenot----
                venue  freq
0         Bridal Shop   0.1
1          Donut Shop   0.1
2       Deli / Bodega   0.1
3  Italian Restaurant   0.1
4    Asian Restaurant   0.1


----Hunters Point----
                  venue  freq
0    Italian Restaurant   0.1
1                  Café   0.1
2           Yoga Studio   0.0
3  Pakistani Restaurant   0.0
4    Photography Studio   0.0


----Hunts Point----
                venue  freq
0           Juice Bar   0.1
1           BBQ Joint   0.1
2       Grocery Store   0.1
3        Gourmet Shop   0.1
4  Spanish Restaurant   0.1


----Inwood----
                venue  freq
0  

                 venue  freq
0  American Restaurant   0.1
1            Bookstore   0.1
2          Coffee Shop   0.1
3                 Park   0.1
4          Yoga Studio   0.0


----Morris Heights----
                  venue  freq
0    Spanish Restaurant   0.2
1            Food Truck   0.2
2           Bus Station   0.1
3           Pizza Place   0.1
4  Fast Food Restaurant   0.1


----Morris Park----
                           venue  freq
0                    Pizza Place   0.2
1                         Bakery   0.1
2                  Deli / Bodega   0.1
3                   Burger Joint   0.1
4  Paper / Office Supplies Store   0.0


----Morrisania----
                  venue  freq
0        Discount Store   0.2
1         Metro Station   0.1
2            Donut Shop   0.1
3  Fast Food Restaurant   0.1
4           Bus Station   0.1


----Mott Haven----
                venue  freq
0  Spanish Restaurant   0.1
1         Pizza Place   0.1
2                 Gym   0.1
3   Paella Restaurant   0.0
4  

                venue  freq
0  Chinese Restaurant   0.2
1                Bank   0.1
2              Bakery   0.1
3        Optical Shop   0.0
4     Organic Grocery   0.0


----Queensbridge----
                venue  freq
0               Hotel   0.3
1           Hotel Bar   0.1
2           Roof Deck   0.1
3  Spanish Restaurant   0.1
4      Scenic Lookout   0.1


----Randall Manor----
           venue  freq
0       Bus Stop   0.3
1           Park   0.1
2     Bagel Shop   0.1
3    Pizza Place   0.1
4  Deli / Bodega   0.1


----Ravenswood----
                  venue  freq
0    Italian Restaurant   0.1
1         Grocery Store   0.1
2           Yoga Studio   0.0
3  Pakistani Restaurant   0.0
4    Photography Studio   0.0


----Red Hook----
                           venue  freq
0             Seafood Restaurant   0.1
1                           Park   0.1
2                    Art Gallery   0.1
3                    Yoga Studio   0.0
4  Paper / Office Supplies Store   0.0


----Rego Park----
     

                venue  freq
0                Park   1.0
1         Yoga Studio   0.0
2            Pie Shop   0.0
3  Photography Studio   0.0
4            Pharmacy   0.0


----Tompkinsville----
                  venue  freq
0               Brewery   0.1
1         Deli / Bodega   0.1
2  Pakistani Restaurant   0.0
3             Piano Bar   0.0
4    Photography Studio   0.0


----Tottenville----
                    venue  freq
0      Mexican Restaurant   0.1
1                Bus Stop   0.1
2  Thrift / Vintage Store   0.1
3      Italian Restaurant   0.1
4          Cosmetics Shop   0.1


----Travis----
                  venue  freq
0           Pizza Place   0.1
1         Bowling Alley   0.1
2                  Park   0.1
3  Gym / Fitness Center   0.1
4          Home Service   0.1


----Tribeca----
                venue  freq
0         Yoga Studio   0.0
1            Pie Shop   0.0
2  Photography Studio   0.0
3            Pharmacy   0.0
4           Pet Store   0.0


----Tudor City----
          

In [56]:
# function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

# create columns according to number of top venues
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create new dataframes
london_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
london_neighborhoods_venues_sorted['Neighborhood'] = london_grouped['Neighborhood']
for ind in np.arange(london_grouped.shape[0]):
    london_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)
    
toronto_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
toronto_neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
for ind in np.arange(toronto_grouped.shape[0]):
    toronto_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)   
    
newyork_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
newyork_neighborhoods_venues_sorted['Neighborhood'] = newyork_grouped['Neighborhood']
for ind in np.arange(newyork_grouped.shape[0]):
    newyork_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)   

In [61]:
# set number of clusters
kclusters = 5

# run k-means clustering
london_grouped_clustering = london_grouped.drop('Neighborhood', 1)
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
newyork_grouped_clustering = newyork_grouped.drop('Neighborhood', 1)

london_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)
toronto_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
newyork_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(newyork_grouped_clustering)

# add clustering labels
london_neighborhoods_venues_sorted=london_neighborhoods_venues_sorted.drop(columns=["Cluster Labels"])
london_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', london_kmeans.labels_)
toronto_neighborhoods_venues_sorted=toronto_neighborhoods_venues_sorted.drop(columns=["Cluster Labels"])
toronto_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', toronto_kmeans.labels_)
newyork_neighborhoods_venues_sorted=newyork_neighborhoods_venues_sorted.drop(columns=["Cluster Labels"])
newyork_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', newyork_kmeans.labels_)

london_merged = london
toronto_merged = toronto
newyork_merged = newyork

# merge _grouped with _data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(london_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = toronto_merged.join(toronto_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
newyork_merged = newyork_merged.join(newyork_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

london_merged=london_merged.dropna(axis=0)
toronto_merged=toronto_merged.dropna(axis=0)
newyork_merged=newyork_merged.dropna(axis=0)

In [62]:
# create map
map_clusters = folium.Map(location=[51.30, -0.50], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['Neighborhood'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [63]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Bethnal Green,Bow,Bromley by Bow,Poplar,Stepne...",0.0,Pub,Grocery Store,Hotel,Bar,Light Rail Station,Café,Chinese Restaurant,Locksmith,Gym,Park
5,"Barking,Beckton,East Ham",0.0,Grocery Store,Bar,Park,Nature Preserve,Café,Film Studio,Exhibit,Falafel Restaurant,Farm,Farmers Market
8,"Hackney,Homerton,Poplar",0.0,Pub,Café,Yoga Studio,Coffee Shop,Bus Stop,Garden,Gastropub,Beer Store,Park,Train Station
10,"Leyton,Leytonstone,Wanstead",0.0,Pub,Grocery Store,Coffee Shop,Pizza Place,Fast Food Restaurant,Café,Dim Sum Restaurant,Clothing Store,Mediterranean Restaurant,Bakery
12,"Plaistow,West Ham",0.0,Pub,Gym,Gym / Fitness Center,Bus Station,Café,Flower Shop,Flea Market,Food,Fishing Store,Fish Market
23,"East Finchley,Hornsey",0.0,Café,Pub,Coffee Shop,Platform,Italian Restaurant,Cricket Ground,Seafood Restaurant,Metro Station,Farmers Market,Bus Stop
26,"Highbury,Islington,Tottenham",0.0,Café,Thai Restaurant,Fish & Chips Shop,Cheese Shop,Restaurant,Coffee Shop,Pub,Bookstore,Middle Eastern Restaurant,Tapas Restaurant
27,"Hampstead,Highgate,Hornsey,St. Pancras",0.0,Pub,Pizza Place,Park,Japanese Restaurant,Café,Grocery Store,Deli / Bodega,Coffee Shop,Chinese Restaurant,Seafood Restaurant
31,"Hornsey,Muswell Hill,Wood Green",0.0,Café,Coffee Shop,Pizza Place,Grocery Store,Pub,Deli / Bodega,Italian Restaurant,Bakery,French Restaurant,Cheese Shop
36,"Seven Sisters,South Tottenham",0.0,Grocery Store,Metro Station,Pub,Hostel,Hardware Store,Café,Chinese Restaurant,Fast Food Restaurant,Event Space,Exhibit


In [64]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Mill Hill,1.0,Park,Fish & Chips Shop,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market


In [65]:
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"Chingford,Highams Park,Stepney",2.0,Grocery Store,American Restaurant,Bus Stop,Xinjiang Restaurant,Fish Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio
6,"East Ham,Forest Gate,Upton Park,Wanstead,West Ham",2.0,Grocery Store,Fast Food Restaurant,Pub,Bus Stop,Fish & Chips Shop,Bakery,Restaurant,Train Station,Indian Restaurant,Moving Target
11,"East Ham,Manor Park,Wanstead",2.0,Indian Restaurant,Gym / Fitness Center,Fast Food Restaurant,Restaurant,Xinjiang Restaurant,Film Studio,Event Space,Exhibit,Falafel Restaurant,Farm
32,"Friern Barnet,New Southgate",2.0,Grocery Store,Pet Store,Toy / Game Store,Train Station,Hardware Store,Fast Food Restaurant,Electronics Store,Coffee Shop,Shopping Plaza,Furniture / Home Store
45,"Cricklewood,Dollis Hill,Hampstead,Hendon,Neasd...",2.0,Grocery Store,Fast Food Restaurant,Coffee Shop,Hotel,Breakfast Spot,Italian Restaurant,Warehouse Store,Bagel Shop,Café,Gym / Fitness Center
53,"Acton,Hammersmith,Harlesden,Kensal Green,Kensa...",2.0,Grocery Store,Fast Food Restaurant,Pizza Place,Pub,Sandwich Place,Restaurant,Coffee Shop,Café,Plaza,Warehouse Store
82,Thamesmead,2.0,Fast Food Restaurant,Pet Store,Furniture / Home Store,Warehouse Store,Supermarket,Xinjiang Restaurant,Exhibit,Falafel Restaurant,Farm,Farmers Market
84,"Brixton,Lambeth,Streatham Hill,Wandsworth",2.0,Grocery Store,Vegetarian / Vegan Restaurant,Bus Stop,Fast Food Restaurant,Caribbean Restaurant,Dry Cleaner,Park,Coffee Shop,Convenience Store,Pub


In [66]:
london_merged.loc[london_merged['Cluster Labels'] == 3, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Aldgate,Bethnal Green,City of London,Mile End,...",3.0,Pub,Hotel,Sandwich Place,Indian Restaurant,Grocery Store,Coffee Shop,Ice Cream Shop,Burger Joint,Bus Stop,Turkish Restaurant
1,"Bethnal Green,Shoreditch,Stepney",3.0,Coffee Shop,Café,Pub,Cocktail Bar,Hotel,Bar,Restaurant,Flower Shop,Park,Pizza Place
4,"Clapton,Hackney",3.0,Grocery Store,Park,Coffee Shop,Burger Joint,Garden,Gym / Fitness Center,Cocktail Bar,Train Station,Beer Bar,Turkish Restaurant
7,"Dalston,Hackney,Shoreditch",3.0,Pub,Restaurant,Gym / Fitness Center,Café,Bakery,Yoga Studio,Pool,Park,Coffee Shop,Burger Joint
9,Leyton,3.0,Convenience Store,Cricket Ground,Coffee Shop,Grocery Store,Farm,Train Station,Park,Hotel,Ice Cream Shop,Flower Shop
13,"Isle of Dogs,Millwall,Poplar,Stepney",3.0,Coffee Shop,Italian Restaurant,Burger Joint,Plaza,Sushi Restaurant,Park,Shopping Mall,Food Truck,Pizza Place,Gift Shop
14,"Canning Town,Hackney,Leyton,Poplar,Stratford,W...",3.0,Pub,Hotel,Sandwich Place,Coffee Shop,Bookstore,Bar,General Entertainment,Café,Fast Food Restaurant,Supermarket
15,"Canning Town,Docklands,East Ham,North Woolwich...",3.0,Hotel,Chinese Restaurant,Gym,Pub,Coffee Shop,English Restaurant,Restaurant,Salad Place,Tapas Restaurant,Sandwich Place
16,"Leyton,Walthamstow",3.0,Coffee Shop,Grocery Store,Pub,Pizza Place,Café,Indian Restaurant,Portuguese Restaurant,Burger Joint,Multiplex,Museum
17,"South Woodford,Wanstead,Woodford",3.0,Café,Bar,Coffee Shop,Chinese Restaurant,Italian Restaurant,Supermarket,Grocery Store,Yoga Studio,Building,Thai Restaurant


In [67]:
london_merged.loc[london_merged['Cluster Labels'] == 4, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,"Chislehurst,Eltham,Mottingham,Woolwich",4.0,Golf Course,Xinjiang Restaurant,Fish & Chips Shop,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market


In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge,Malvern",0.0,Print Shop,Fast Food Restaurant,Dog Run,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store
2,"Guildwood,Morningside,West Hill",0.0,Medical Center,Electronics Store,Pizza Place,Spa,Rental Car Location,Intersection,Mexican Restaurant,Breakfast Spot,Diner,Dessert Shop
3,Woburn,0.0,Coffee Shop,Korean Restaurant,Soccer Field,College Gym,College Rec Center,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,College Cafeteria
4,Cedarbrae,0.0,Fried Chicken Joint,Hakka Restaurant,Athletics & Sports,Bakery,Thai Restaurant,Bank,Caribbean Restaurant,Diner,Department Store,Dessert Shop
5,Scarborough Village,0.0,Playground,Cosmetics Shop,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
6,"East Birchmount Park,Ionview,Kennedy Park",0.0,Bus Station,Discount Store,Coffee Shop,Department Store,Hobby Shop,Convenience Store,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner
7,"Clairlea,Golden Mile,Oakridge",0.0,Bakery,Bus Line,Park,Bus Station,Fast Food Restaurant,Metro Station,Intersection,Soccer Field,Concert Hall,Construction & Landscaping
8,"Cliffcrest,Cliffside,Scarborough Village West",0.0,Motel,American Restaurant,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
9,"Birch Cliff,Cliffside West",0.0,General Entertainment,Café,College Stadium,Skating Rink,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
10,"Dorset Park,Scarborough Town Centre,Wexford He...",0.0,Indian Restaurant,Chinese Restaurant,Pet Store,Latin American Restaurant,Vietnamese Restaurant,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant


In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Agincourt North,L'Amoreaux East,Milliken,Steel...",1.0,Gym,Park,Playground,Dog Run,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
25,Parkwoods,1.0,Park,Fast Food Restaurant,Food & Drink Shop,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
40,East Toronto,1.0,Park,Coffee Shop,Convenience Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
50,Rosedale,1.0,Park,Playground,Trail,Women's Store,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
74,Caledonia-Fairbanks,1.0,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy,College Rec Center,College Stadium,Eastern European Restaurant,College Cafeteria,Dumpling Restaurant
90,"The Kingsway,Montgomery Road,Old Mill North",1.0,River,Park,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
98,Weston,1.0,Park,Convenience Store,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run


In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,"Silver Hills,York Mills",2.0,Cafeteria,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Dance Studio


In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,York Mills West,3.0,Bank,Park,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
94,"Cloverdale,Islington,Martin Grove,Princess Gar...",3.0,Bank,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Deli / Bodega


In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek,Rouge Hill,Port Union",4.0,Bar,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store,Deli / Bodega


In [73]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 0, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,River,High School,Playground,Plaza,Women's Store,Farm,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service
4,Riverdale,Food Truck,Gym,Playground,Plaza,Park,Home Service,Bank,Falafel Restaurant,English Restaurant,Ethiopian Restaurant
9,Williamsbridge,Bar,Caribbean Restaurant,Soup Place,Nightclub,Women's Store,Fast Food Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
12,City Island,Harbor / Marina,Seafood Restaurant,Thrift / Vintage Store,Ice Cream Shop,Bar,Diner,Park,Bank,Liquor Store,Pharmacy
22,Port Morris,Brewery,Deli / Bodega,Distillery,Peruvian Restaurant,Latin American Restaurant,Spanish Restaurant,Flower Shop,Furniture / Home Store,Music Venue,Food Truck
24,Hunts Point,Shipping Store,Juice Bar,Bank,Grocery Store,BBQ Joint,Seafood Restaurant,Gourmet Shop,Pizza Place,Farmers Market,Café
27,Clason Point,Park,Scenic Lookout,Grocery Store,Boat or Ferry,Bus Stop,South American Restaurant,Pool,Women's Store,Event Service,Event Space
46,Bay Ridge,Italian Restaurant,Spa,Pizza Place,Greek Restaurant,Bar,Bagel Shop,American Restaurant,Playground,Thai Restaurant,Chinese Restaurant
49,Greenpoint,Bar,Pizza Place,Coffee Shop,Cocktail Bar,Café,Yoga Studio,Bakery,French Restaurant,Mexican Restaurant,Boutique
51,Brighton Beach,Eastern European Restaurant,Restaurant,Beach,Gourmet Shop,Bank,Mobile Phone Shop,Sushi Restaurant,Fast Food Restaurant,Russian Restaurant,Supplement Shop


In [74]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 1, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
192,Somerville,Park,Women's Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant
203,Todt Hill,Park,Women's Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant


In [75]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 2, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Laundromat,Ice Cream Shop,Pharmacy,Dessert Shop,Donut Shop,Sandwich Place,Caribbean Restaurant,Food Truck,Event Space,Exhibit
1,Co-op City,Bus Station,Fast Food Restaurant,Baseball Field,Fried Chicken Joint,Discount Store,Gift Shop,Park,Grocery Store,Basketball Court,Mattress Store
2,Eastchester,Caribbean Restaurant,Bus Station,Metro Station,Diner,Deli / Bodega,Bus Stop,Chinese Restaurant,Bakery,Seafood Restaurant,Fast Food Restaurant
5,Kingsbridge,Pizza Place,Bar,Supermarket,Sandwich Place,Discount Store,Mexican Restaurant,Spanish Restaurant,Bakery,Latin American Restaurant,Pharmacy
6,Marble Hill,Coffee Shop,Discount Store,Kids Store,Tennis Stadium,Shoe Store,Big Box Store,Gym,Bank,Seafood Restaurant,Pharmacy
7,Woodlawn,Deli / Bodega,Playground,Pizza Place,Pub,Moving Target,Bar,Bakery,Donut Shop,Train Station,Park
8,Norwood,Pizza Place,Park,Pharmacy,American Restaurant,Bank,Food Truck,Mobile Phone Shop,Sandwich Place,Liquor Store,Fast Food Restaurant
10,Baychester,Electronics Store,Baseball Field,Discount Store,Mattress Store,Gym / Fitness Center,Mexican Restaurant,Bank,Pet Store,Fast Food Restaurant,Boat or Ferry
11,Pelham Parkway,Deli / Bodega,Pizza Place,Italian Restaurant,Chinese Restaurant,Bus Station,Performing Arts Venue,Metro Station,Smoke Shop,Track,Eye Doctor
13,Bedford Park,Chinese Restaurant,Pizza Place,Deli / Bodega,Diner,Sandwich Place,Supermarket,Mexican Restaurant,Pharmacy,Dry Cleaner,Donut Shop


In [76]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 3, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
255,Emerson Hill,Construction & Landscaping,Moving Target,Women's Store,Filipino Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant
257,Howland Hook,Harbor / Marina,Moving Target,Women's Store,Field,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory


In [77]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 4, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
198,New Brighton,Bus Stop,Deli / Bodega,Park,Botanical Garden,Playground,Discount Store,Convenience Store,Bowling Alley,Eye Doctor,Factory
212,Oakwood,Bar,Bus Stop,Women's Store,Filipino Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant
224,Park Hill,Bus Stop,Hotel,Gym / Fitness Center,Athletics & Sports,Coffee Shop,Fast Food Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
227,Arlington,Bus Stop,Furniture / Home Store,Coffee Shop,Filipino Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant
256,Randall Manor,Bus Stop,Bagel Shop,Park,Deli / Bodega,Pizza Place,Playground,Fast Food Restaurant,Event Service,Event Space,Exhibit
285,Willowbrook,Bus Stop,Bagel Shop,Intersection,Chinese Restaurant,Deli / Bodega,Pizza Place,Spa,Farmers Market,Ethiopian Restaurant,Event Service
305,Fox Hills,Bus Stop,Food,Grocery Store,Sandwich Place,Women's Store,Fast Food Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
